In [1]:
print("Saad")

Saad


In [2]:
 %pwd

'd:\\ChatBotProjects\\MedicalChatBot\\research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'd:\\ChatBotProjects\\MedicalChatBot'

In [39]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings,ChatHuggingFace,HuggingFaceEndpoint

In [6]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
def text_split(extracted_data):
    text_split=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_split.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

5859


In [11]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings=download_hugging_face_embeddings()

In [18]:
query_result=embeddings.embed_query("Hello_world")
print('Length',len(query_result))

Length 384


In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=os.environ.get('HUGGINGFACEHUB_API_TOKEN')

In [25]:
import os
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY

In [22]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medibot"
pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-c3mu5un.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [26]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [27]:
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [28]:
docsearch

In [30]:
retriever=docsearch.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [31]:
retriver_docs=retriever.invoke('What is Ace')

In [32]:
retriver_docs

[Document(id='0c95808e-386d-4cc2-8af0-dc741706314b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 218.0, 'page_label': '219', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='animal, usually delivered through a bite or a sting.\nACE is to help regulate arterial pressure by converting\nangiotensin I to angiotensin II.\nPurpose\nThe ACE test is used primarily to detect and monitor\nthe clinical course of sarcoidosis (a disease that affects\nmany organs, especially the lungs), to differentiate\nbetween sarcoidosis and similar diseases, and to delin-\neate between active and inactive sarcoid disease. Elevat-\ned ACE levels are also found in a number of other condi-'),
 Document(id='80321e80-edc1-4b3d-aabc-991495511f00', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 218.0, 'p

In [33]:
from langchain_huggingface import HuggingFaceEndpoint
llm=HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.3',
    task='text-generation'
)
model=ChatHuggingFace(llm=llm)


In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [35]:
system_prompt=(
    'You are an assistant for question_answering tasks created by Muhammad Saad,'
    'Use the following pieces of retrieved context to answer '
    'the question. if you do not know the answer, say that you  '
    'do not know the answer because you are only medical assisatant not for other things'
    'use three sentences maximum and keep the answer concise '
    '\n\n'
    '{context}'
)
prompt=ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human','{input}')
    ]
)

In [36]:
question_answer_chain=create_stuff_documents_chain(model,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [38]:
response=rag_chain.invoke({'input':'What is acne?'})
print(response['answer'])

 Acne is a skin disorder characterized by inflammation of the sebaceous glands. It can affect various areas of the body, including the face.
